In [1]:
# Among high and low quality items, try to identify useful patterns in text complexity metrics
import pandas as pd
import torch
import numpy as np
from copy import deepcopy
from evaluate import load
from readability import Readability
# CPS
train = pd.read_csv("/home/aml7990/Code/creativity-item-generation/item_evaluation/cleaned_datasets/CPS_train.csv")
test = pd.read_csv("/home/aml7990/Code/creativity-item-generation/item_evaluation/cleaned_datasets/CPS_test.csv")
df = pd.concat([train, test])

# Bloom Social Science items
# df = pd.read_json("/home/aml7990/Code/creativity-item-generation/item_evaluation/cleaned_datasets/AEQG-SocialSciences-Bloom.json", orient="records")

/home/aml7990/miniconda3/envs/AIG-CUDA-12.0/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df.head(1)

,creative_scenario_round_4,complexity_aggregrate,difficulty_aggregrate
0,Noah is a high school student who loves to gar...,3,2


In [22]:
len(df)

456

In [23]:
df["BloomsSkill"].value_counts()

BloomsSkill
2    95
1    88
4    87
5    70
6    65
3    51
Name: count, dtype: int64

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct").to("cuda:0")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.96it/s]


In [4]:
from transformers import AutoModelForSequenceClassification
readability_tokenizer= AutoTokenizer.from_pretrained("agentlans/deberta-v3-xsmall-readability-v2")
readability_model = AutoModelForSequenceClassification.from_pretrained("agentlans/deberta-v3-xsmall-readability-v2").to("cuda:1")
def readability(text):
    """Processes the text using the model and returns its logits.
    In this case, it's reading grade level in years of education
    (the higher the number, the harder it is to read the text)."""
    inputs = readability_tokenizer(text, return_tensors="pt", truncation=True, padding=True).to("cuda:1")
    with torch.no_grad():
        logits = readability_model(**inputs).logits.squeeze().cpu()
    return logits.tolist()

In [5]:
# taken from the official hf implementation: https://huggingface.co/spaces/evaluate-metric/perplexity/blob/main/perplexity.py
from evaluate import logging
from torch.nn import CrossEntropyLoss
def compute_perplexity(
        predictions, model, tokenizer, batch_size: int = 16, add_start_token: bool = True, device=None, max_length=None
    ):

        if device is not None:
            assert device in ["gpu", "cpu", "cuda"], "device should be either gpu or cpu."
            if device == "gpu":
                device = "cuda"
        else:
            device = "cuda" if torch.cuda.is_available() else "cpu"

        # if batch_size > 1 (which generally leads to padding being required), and
        # if there is not an already assigned pad_token, assign an existing
        # special token to also be the padding token
        if tokenizer.pad_token is None and batch_size > 1:
            existing_special_tokens = list(tokenizer.special_tokens_map_extended.values())
            # check that the model already has at least one special token defined
            assert (
                len(existing_special_tokens) > 0
            ), "If batch_size > 1, model must have at least one special token to use for padding. Please use a different model or set batch_size=1."
            # assign one of the special tokens to also be the pad token
            tokenizer.add_special_tokens({"pad_token": existing_special_tokens[0]})

        if add_start_token and max_length:
            # leave room for <BOS> token to be added:
            assert (
                tokenizer.bos_token is not None
            ), "Input model must already have a BOS token if using add_start_token=True. Please use a different model, or set add_start_token=False"
            max_tokenized_len = max_length - 1
        else:
            max_tokenized_len = max_length

        encodings = tokenizer(
            predictions,
            add_special_tokens=False,
            padding=True,
            truncation=True if max_tokenized_len else False,
            max_length=max_tokenized_len,
            return_tensors="pt",
            return_attention_mask=True,
        ).to(device)

        encoded_texts = encodings["input_ids"]
        attn_masks = encodings["attention_mask"]

        # check that each input is long enough:
        if add_start_token:
            assert torch.all(torch.ge(attn_masks.sum(1), 1)), "Each input text must be at least one token long."
        else:
            assert torch.all(
                torch.ge(attn_masks.sum(1), 2)
            ), "When add_start_token=False, each input text must be at least two tokens long. Run with add_start_token=True if inputting strings of only one token, and remove all empty input strings."

        ppls = []
        loss_fct = CrossEntropyLoss(reduction="none")

        for start_index in logging.tqdm(range(0, len(encoded_texts), batch_size)):
            end_index = min(start_index + batch_size, len(encoded_texts))
            encoded_batch = encoded_texts[start_index:end_index]
            attn_mask = attn_masks[start_index:end_index]

            if add_start_token:
                bos_tokens_tensor = torch.tensor([[tokenizer.bos_token_id]] * encoded_batch.size(dim=0)).to(device)
                encoded_batch = torch.cat([bos_tokens_tensor, encoded_batch], dim=1)
                attn_mask = torch.cat(
                    [torch.ones(bos_tokens_tensor.size(), dtype=torch.int64).to(device), attn_mask], dim=1
                )

            labels = encoded_batch

            with torch.no_grad():
                out_logits = model(encoded_batch, attention_mask=attn_mask).logits

            shift_logits = out_logits[..., :-1, :].contiguous()
            shift_labels = labels[..., 1:].contiguous()
            shift_attention_mask_batch = attn_mask[..., 1:].contiguous()

            perplexity_batch = torch.exp(
                (loss_fct(shift_logits.transpose(1, 2), shift_labels) * shift_attention_mask_batch).sum(1)
                / shift_attention_mask_batch.sum(1)
            )

            ppls += perplexity_batch.tolist()

        return {"perplexities": ppls, "mean_perplexity": np.mean(ppls)}

In [6]:
# define the metrics we'll use
# perplexity
df["perplexity"] = df["creative_scenario_round_4"].apply(lambda x: compute_perplexity(predictions=[x], model=model, tokenizer=tokenizer)["mean_perplexity"])

  0%|          | 0/1 [00:00<?, ?it/s]We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


100%|██████████| 1/1 [00:00<00:00,  4.94it/s]


In [7]:
# burstiness
# implementation taken from: https://medium.com/@jhanwarsid/human-contentanalysing-perplexity-and-burstiness-in-ai-vs-human-text-df70fdcc5525

def calculate_burstiness(text):
    sentences = text.split('. ')
    sentence_lengths = [len(tokenizer.encode(sentence)) for sentence in sentences if sentence]
    burstiness = np.std(sentence_lengths)
    return burstiness


df["burstiness"] = df["creative_scenario_round_4"].apply(lambda x: calculate_burstiness(x))

In [8]:
# fano factor, similar to burstiness but captures both differnces in mean and variance
# implementation taken from: https://medium.com/@jhanwarsid/human-contentanalysing-perplexity-and-burstiness-in-ai-vs-human-text-df70fdcc5525

def calculate_fano_factor(text):
    sentences = text.split('. ')
    sentence_lengths = [len(tokenizer.encode(sentence)) for sentence in sentences if sentence]
    mean_length = np.mean(sentence_lengths)
    variance = np.var(sentence_lengths)
    fano_factor = variance / mean_length if mean_length > 0 else 0
    return fano_factor


df["fano_factor"] = df["creative_scenario_round_4"].apply(lambda x: calculate_fano_factor(x))

In [9]:
# Automated readability index (doesn't work for the bloom dataset)
df["ARI"] = df["creative_scenario_round_4"].apply(lambda x: Readability(x).ari().score)
# df["readability"] = df["creative_scenario_round_4"].apply(lambda x: readability(x))

In [10]:
# token type ratio
# implementation from: https://medium.com/@rajeswaridepala/empirical-laws-ttr-cc9f826d304d
# I have tweaked it slightly to better handle punctuation
def calculate_ttr(text):

    # tokens are nothing but words in the sentence
    text = text.replace(".", "").replace("?", "").replace("!", "").replace(",", "")
    words = text.split()

    # type
    unique_words = set(words)

    # The TTR value is always between 0 and 1, where higher values indicate greater lexical diversity.
    ttr = len(unique_words) / len(words)

    return ttr

df["ttr"] = df["creative_scenario_round_4"].apply(lambda x: calculate_ttr(x))

In [11]:
# an alternative approach is to use text embeddings instead of the information theoretic features
import torch.nn.functional as F
from torch import Tensor
from transformers import AutoModel, AutoTokenizer
from sentence_transformers.util import cos_sim
def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

model_path = 'thenlper/gte-small'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).to("cuda:2")
batch_dict = tokenizer(df["creative_scenario_round_4"].to_list(), max_length=8192, padding=True, truncation=True, return_tensors='pt').to("cuda:2")

outputs = model(**batch_dict)
embeddings = outputs.last_hidden_state[:, 0]
 
# (Optionally) normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings @ embeddings.T) * 100
scores = list(scores[0].detach().cpu())
print(scores)

[tensor(100.0000), tensor(95.3724), tensor(95.3592), tensor(95.0047), tensor(94.9642), tensor(95.4893), tensor(95.7182), tensor(95.1296), tensor(95.0241), tensor(96.2331), tensor(94.8144), tensor(95.3802), tensor(94.8896), tensor(95.8391), tensor(95.7969), tensor(94.7606), tensor(95.2712), tensor(95.0924), tensor(95.3448), tensor(95.3293), tensor(95.5455), tensor(95.5215), tensor(95.3816), tensor(95.2899), tensor(95.4145), tensor(93.8860), tensor(95.2848), tensor(94.5221), tensor(94.7435), tensor(95.3309), tensor(94.8927), tensor(95.6405), tensor(95.4657), tensor(95.5966), tensor(94.9863), tensor(93.9462), tensor(95.1080), tensor(93.8950), tensor(94.3849), tensor(94.9762), tensor(95.1565), tensor(94.1645), tensor(95.2614), tensor(95.5566), tensor(94.4896), tensor(94.8634), tensor(94.8949), tensor(94.7370), tensor(95.6271), tensor(95.0108), tensor(95.3081), tensor(94.9933), tensor(94.5662), tensor(94.7024), tensor(95.3438), tensor(94.7369), tensor(94.4229), tensor(94.8200), tensor(95.62

In [12]:
scores = [float(x) for x in scores]

In [13]:
df["mean_similarity"] = scores

In [14]:
df.head(5)

,creative_scenario_round_4,complexity_aggregrate,difficulty_aggregrate,perplexity,burstiness,fano_factor,ARI,ttr,mean_similarity
0,Noah is a high school student who loves to gar...,3,2,3.365829,7.946099,3.215488,6.294411,0.562842,99.999992
1,Amy is a barista at Lily's Cafe. She has been ...,2,2,3.763808,7.803133,3.149425,6.182933,0.580000,95.372375
2,Amy works at Lily's Cafe. She has been working...,2,3,3.499628,5.493406,1.913696,5.235199,0.573034,95.359215
3,Lucas has been working at the local bookstore ...,2,1,3.466565,4.887156,1.451532,6.491390,0.567742,95.004692
4,Hannah was working at the library when she not...,3,2,5.651710,3.574473,0.817125,6.201779,0.575540,94.964203


In [15]:
# apply standard scaler to all features

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df[
    ["perplexity", "burstiness", "ARI", "ttr", "fano_factor", "mean_similarity"]
] = scaler.fit_transform(
    df[
        [
            "perplexity",
            "burstiness",
            "ARI",
            "ttr",
            "fano_factor",
            "mean_similarity",
        ]
    ]
)
df.head(3)

,creative_scenario_round_4,complexity_aggregrate,difficulty_aggregrate,perplexity,burstiness,fano_factor,ARI,ttr,mean_similarity
0,Noah is a high school student who loves to gar...,3,2,-0.781108,1.061222,1.131066,-0.943893,-0.112595,6.496502
1,Amy is a barista at Lily's Cafe. She has been ...,2,2,-0.565861,0.974678,1.068521,-1.004129,0.175056,0.343908
2,Amy works at Lily's Cafe. She has been working...,2,3,-0.708742,-0.423512,-0.101421,-1.516228,0.058270,0.326411


In [14]:
# save the pre-computed features
# df.to_csv("/home/aml7990/Code/creativity-item-generation/item_evaluation/linear_regression/llama-3-8b-instruct-full-set.csv")

In [21]:
import statsmodels.api as sm
import numpy as np
from scipy.stats import bootstrap
from statistics import median

n_samples = 1000
frac = 0.9
results = []
# we use a boostrapping approach to fit a CI on the adjusted r squared

for sample in range(n_samples):
    sample = df.sample(frac=frac, replace=True)
    Y = sample["complexity_aggregrate"].to_numpy()
    X = sample[["perplexity", "burstiness", "ARI", "ttr", "fano_factor", "mean_similarity"]].to_numpy()
    # X = sample[["perplexity", "readability", "ttr", "fano_factor"]].to_numpy()
    
    X = sm.add_constant(X)
    model = sm.OLS(Y, X).fit()
    results.append(model.rsquared_adj)

# calculate 95% bootstrapped confidence interval for median
bootstrap_ci = bootstrap((results,), np.median, confidence_level=0.95, random_state=1, method='percentile')

print(bootstrap_ci.confidence_interval)
print(median(results))




# [["perplexity", "readability", "ttr", "fano_factor"]]



ConfidenceInterval(low=0.1968258677175594, high=0.20568798167434388)
0.20164367938065092


In [75]:
results

[0.10725822571215704,
 0.1912212331814499,
 0.28301260164928355,
 0.2721416611099976,
 0.17569914068600057,
 0.27006827121233046,
 0.19179706201851898,
 0.17488301675236417,
 0.15078144671466343,
 0.1609815007722727,
 0.2587854275047806,
 0.16174562309882456,
 0.20244327692221042,
 0.19286394438249377,
 0.24339396410465197,
 0.24490271198687164,
 0.214330891612182,
 0.14374607748954893,
 0.21486755696440563,
 0.20205993375277786,
 0.15919758728535593,
 0.12507550682038737,
 0.16689922749508257,
 0.254780198332734,
 0.19427611634991637,
 0.14561229025965505,
 0.175720335802516,
 0.16050196333695332,
 0.12434041014179098,
 0.20997157179807402,
 0.18021860093997322,
 0.2746495430496577,
 0.18466307291622008,
 0.16670127286940295,
 0.2794849614286683,
 0.2687881626390407,
 0.16712147666743893,
 0.21740844838981832,
 0.15750504428596446,
 0.26120406828473663,
 0.12907781523661277,
 0.14801614061223245,
 0.2758761315902909,
 0.28354235337980926,
 0.2478619763815303,
 0.1891395307020718,
 0.2

In [71]:
Y = df["difficulty_aggregrate"].to_numpy()
X = df[["readability"]].to_numpy()

# [["perplexity","burstiness", "fano_factor", "ARI", "ttr"]]

X = sm.add_constant(X)
model = sm.OLS(Y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     3.398
Date:                Thu, 17 Oct 2024   Prob (F-statistic):             0.0669
Time:                        13:28:59   Log-Likelihood:                -194.25
No. Observations:                 183   AIC:                             392.5
Df Residuals:                     181   BIC:                             398.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.3777      0.306      4.495      0.0